In [1]:
import os
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
class argument:
    def __init__(self):
        self.dataset_name = None # 'wikitext'
        # self.dataset_config_name = 'wikitext-2-raw-v1'
        self.output_dir = './logs/' 
        self.seed = 5555
        self.learning_rate = 5e-5
        self.block_size = 1024
        self.do_ref_model = False
        self.hidden_size = 768
        self.beta = 1
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False
        
        self.per_device_train_batch_size = 4
        self.per_device_eval_batch_size = 4
        self.gradient_accumulation_steps = 8
        
        # self.eval_steps = 50
        self.do_ref_model = False
        self.lr_scheduler_type = 'linear'

        self.num_train_epochs = 3
        self.max_train_steps = None

        self.preprocessing_num_workers = 1
        self.overwrite_cache = False
        self.weight_decay = 0.0
        self.num_warmup_steps = 0
        
        self.add_canary = True
        self.canary_rep = 50
        self.canary_len = 5
        
        self.add_adapter = False
        self.adapter_reduction = 16
        self.train_head_only = False
        self.train_layer_n_only = None 
        
        
args = argument()

In [3]:
from enum import unique
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import copy 
from sys import path
import sys
# from utils import Logger


import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator, DistributedType
from huggingface_hub import Repository
from transformers import (
#    CONFIG_MAPPING,
#    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

#from torch import AdamW
from transformers.utils.versions import require_version
import datasets
from datasets import load_dataset
from random import shuffle
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import csv
from scipy.stats import skewnorm
from scipy.stats import kstest

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
random.seed(args.seed)

# folder_name = f"canary_{str(args.canary_rep)}_{str(args.canary_len)}_adapter_{args.add_adapter}_head_{args.train_head_only}_layer_{args.train_layer_n_only}_ref_{args.do_ref_model}_maxlen_{args.block_size}_red_{args.adapter_reduction}_model_{args.model_name_or_path}_lr_{args.learning_rate}_epoch_{args.num_train_epochs}_trba_{args.per_device_train_batch_size}_acc_{args.gradient_accumulation_steps}_evba{args.per_device_eval_batch_size}_data_{args.dataset_name}"
folder_name = 'GPT2_SWEP'
directory = "{}/{}".format(args.output_dir,folder_name)
print(directory)
if not os.path.exists(directory):
    os.mkdir(directory)

# log_file = os.path.join(directory, "stdout")

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# if accelerator.is_local_main_process:
    # print("Logging to {}".format(log_file))
    # pass
    
# sys.stdout = Logger(log_file)

./logs//GPT2_SWEP


In [5]:
args.train_data_file = './data/wikitext-2-raw/wiki.train.raw'
args.eval_data_file = './data/wikitext-2-raw/wiki.valid.raw'
args.test_data_file = './data/wikitext-2-raw/wiki.test.raw'
raw_datasets = load_dataset('text', data_files={'train': args.train_data_file ,'test': args.test_data_file ,'validation': args.eval_data_file})
raw_datasets

Found cached dataset text (/home/todsavadt/.cache/huggingface/datasets/text/default-dad1a8437413d60e/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
100%|██████████| 3/3 [00:00<00:00, 2008.13it/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [6]:
import re
# Define the regular expressions
def clean_text(example):
    text = example["text"]
    text = re.sub(r'@\.@', '.', text)
    text = re.sub(r'@,@', ',', text)
    text = re.sub(r'@-@', '-', text)
    example["text"] = text
    return example

# Apply the regular expressions to the train and validation datasets
for split in ["train", "validation", "test"]:
    raw_datasets[split] = raw_datasets[split].map(clean_text)
    
raw_datasets

Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/text/default-dad1a8437413d60e/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-720aa9323723d32f.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/text/default-dad1a8437413d60e/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-b4ecdb89acab5322.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/text/default-dad1a8437413d60e/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-aca3a635fd19f9f1.arrow


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [7]:
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
#    else:
#        config = CONFIG_MAPPING[args.model_type]()
#        logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

## VariationalModel

In [8]:
from math import sqrt

import torch
import torch.nn as nn
# from transformers import BertForQuestionAnswering, ElectraForQuestionAnswering, BertForSequenceClassification
import torch.nn.functional as F

class GaussianKLLoss(nn.Module):
    def __init__(self):
        super(GaussianKLLoss, self).__init__()

    def forward(self, mu1, logvar1, mu2, logvar2):
        numerator = logvar1.exp() + torch.pow(mu1 - mu2, 2)
        fraction = torch.div(numerator, (logvar2.exp()))
        kl = 0.5 * torch.sum(logvar2 - logvar1 + fraction - 1, dim=1)
        return kl.mean(dim=0)

class VariationalGPT2(nn.Module):
    def __init__(self, args, config):
        super(VariationalGPT2, self).__init__()
        self.model_name = args.model_name_or_path
        self.transformer = AutoModelForCausalLM.from_pretrained(
                            args.model_name_or_path,
                            from_tf=bool(".ckpt" in args.model_name_or_path),
                            config=config,
                            )
        
        self.noise_net = nn.Sequential(nn.Linear(args.hidden_size,
                                                 args.hidden_size),
                                       nn.ReLU(),
                                       nn.Linear(args.hidden_size,
                                                 args.hidden_size * 2))
        # config = self.bert_model.config
        self.dropout = config.attn_pdrop  # 0.1

    def forward(self, input_ids,
                attention_mask,
                labels):

        # if start_positions is not None and end_positions is not None:
        embeddings = self.transformer.transformer.wte
        encoder = self.transformer
        with torch.no_grad():
            encoder_inputs = {"input_ids": input_ids,
                              "attention_mask": attention_mask}

            outputs = encoder(**encoder_inputs, output_hidden_states=True)
            hiddens = outputs.hidden_states[0]
        
        # print(hiddens.shape)
        mask = attention_mask.view(-1)
        indices = (mask == 1)
        mu_logvar = self.noise_net(hiddens)
        mu, log_var = torch.chunk(mu_logvar, 2, dim=-1)
        zs = mu + torch.randn_like(mu) * torch.exp(0.5 * log_var)
        noise = zs

        prior_mu = torch.ones_like(mu)
        # If p < 0.5, sqrt makes variance the larger
        prior_var = torch.ones_like(mu) * sqrt(self.dropout / (1-self.dropout))
        prior_logvar = torch.log(prior_var)

        kl_criterion = GaussianKLLoss()
        h = hiddens.size(-1)
        _mu = mu.view(-1, h)[indices]
        _log_var = log_var.view(-1, h)[indices]
        _prior_mu = prior_mu.view(-1, h)[indices]
        _prior_logvar = prior_logvar.view(-1, h)[indices]

        kl = kl_criterion(_mu, _log_var, _prior_mu, _prior_logvar)

        inputs_embeds = self.transformer.transformer.wte(input_ids)
        # print(inputs_embeds.shape)
        # print(noise.shape)
        inputs = {"inputs_embeds": inputs_embeds * noise,
                  "attention_mask": attention_mask,
                  "labels":labels }

        noise_outputs = self.transformer(**inputs)
        noise_loss = noise_outputs[0]

        new_inputs = {"inputs_embeds": inputs_embeds,
                      "attention_mask": attention_mask,
                      "labels":labels }

        outputs = self.transformer(**new_inputs)
        nll = outputs[0] #loss
        loss = 0.5 * (nll + noise_loss)
        return loss, kl, outputs, noise_outputs
                    

In [9]:
# model.transformer.transformer

In [10]:
model = VariationalGPT2(args, config)

In [11]:
# if args.model_name_or_path:
#     model = AutoModelForCausalLM.from_pretrained(
#         args.model_name_or_path,
#         from_tf=bool(".ckpt" in args.model_name_or_path),
#         config=config,
#     )
# else:
#     logger.info("Training new model from scratch")
#     model = AutoModelForCausalLM.from_config(config)

# model.resize_token_embeddings(len(tokenizer))

# # model_ref = copy.deepcopy(model)
# # model_ref.eval()

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#print(model.lm_head)    
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 126.211584


### Add canary

In [13]:
def gen_canary(canary_len,tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [14]:
if args.add_canary:    
    try:
        if 'ptb' in args.dataset_name:
            dict_key = 'sentence'
        else:
            dict_key='text'
    except: 
        dict_key = 'text'
    print("before canary len ", len(raw_datasets['train'][dict_key]))
    canary, canary_ids = gen_canary(args.canary_len, tokenizer)
    for j in range(args.canary_rep):
        raw_datasets['train']=raw_datasets['train'].add_item({dict_key:canary})

    raw_datasets['train'] = raw_datasets['train'].shuffle(seed=args.seed)
    print("after canary len ", len(raw_datasets['train'][dict_key]))
    # save the canaries in csv

    file = open(f'./{directory}/canaries.txt', 'w+')
    file.write(canary)
    file.write('\n')
    file.close()

    file = open(f'./{directory}/fitting_canaries.txt', 'w+')
    
    fitting_canaries_ids = []
    for i in range(5000):
        fit , fit_ids = gen_canary(args.canary_len,tokenizer)
        if fit != canary:
            fitting_canaries_ids.append(fit_ids)
            file.write(fit)
            file.write('\n')
    print(len(fitting_canaries_ids))

before canary len  36718
after canary len  36768
5000


## Preprocessing

In [15]:
# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

with accelerator.main_process_first():
    tokenized_datasets = raw_datasets.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        print(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
    block_size = 1024
else:
    if args.block_size > tokenizer.model_max_length:
        print(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)

In [16]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
    
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

with accelerator.main_process_first():
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        desc=f"Grouping texts in chunks of {block_size}",
    )

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

In [17]:
train_dataset, eval_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 2261
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 233
 }))

In [18]:
small_train_dataset = train_dataset.shuffle(seed=55).select(range(20))
small_eval_dataset = eval_dataset.shuffle(seed=55).select(range(5))
small_train_dataset, small_eval_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 20
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 5
 }))

In [19]:
# Log a few random samples from the training set:
#for index in random.sample(range(len(train_dataset)), 3):
#    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")
args.small_sample = False
if args.small_sample:
    # DataLoaders creation:
    train_dataloader = DataLoader(
        small_train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
    )
    eval_dataloader = DataLoader(
        small_eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
    )
else:
    # DataLoaders creation:
    train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
    )
    eval_dataloader = DataLoader(
        eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
    )

In [20]:
#checking chucking
for i in train_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break
for i in eval_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break

torch.Size([4, 1024]) torch.Size([4, 1024])
torch.Size([4, 1024]) torch.Size([4, 1024])


In [21]:
 # Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# model_ref = accelerator.prepare(
#     model_ref
# )

In [23]:
# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type == DistributedType.TPU:
    model.tie_weights()

# Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
# shorter in multiprocess)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

In [24]:
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 126.211584


In [25]:
# def calculate_exposure(canary_rank):
    # return math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

In [26]:
def get_exposure(fitting, main):

    fitting_params = skewnorm.fit(fitting)
    ks = kstest(fitting, 'skewnorm', fitting_params)

    cdf = skewnorm.cdf(main, fitting_params[0], fitting_params[1], fitting_params[2])

    if cdf == 0.0:
        exposure = 0.0
    else:
        exposure = -1.0*np.log2(cdf)

    return exposure

def get_fit_canary_loss(model, fitting_id, main_id):
    loss_list = []
    for k, v in main_id.items():
        main_id[k] = torch.tensor(v).cuda().unsqueeze(0)

    outputs = model(**main_id)
    loss_main = np.exp(outputs[0].item())
    # loss_main = np.exp(model(**main_id)['loss'].item())

    for sample in fitting_id:
        for k, v in sample.items():
            sample[k] = torch.tensor(v).cuda().unsqueeze(0)
        
        output = model(**sample)
        loss_list.append(np.exp(outputs[0].item()))
        # loss_list.append(np.exp(output.loss.item()))

    return loss_main, loss_list

def gen_canary(canary_len, tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [27]:
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {args.num_train_epochs}")
print(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
print(f"  Total optimization steps = {args.max_train_steps}")

***** Running training *****
  Num examples = 2261
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 213


In [28]:
args.add_canary = False

In [29]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
best_loss = 1000000
best_val_perplexity = float("inf")
save_path = f'models/{model.__class__.__name__}_gpt2_wikitext.pt'
for epoch in range(args.num_train_epochs):
    ##################################################################
    # Train
    ##################################################################
    model.train()
    if accelerator.is_local_main_process:
        print(f"training epoch {epoch+1}")
    for step, batch in enumerate(tqdm(train_dataloader)):
        outputs = model(**batch)
        # loss = outputs.loss
        nll, kl, outputs, outputs_noise = outputs[0], outputs[1], outputs[2], outputs[3]
        loss_str = "NLL: {:.4f}, KL: {:.4f}".format(
                nll.item(), kl.item())
        loss = nll + kl * args.beta
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # progress_bar.update(1)
            completed_steps += 1
            
        if completed_steps >= args.max_train_steps:
            break 
    print(loss_str)
            
    ##################################################################
    # Evaluation
    ##################################################################
    model.eval()
    losses = []

    if args.add_canary:
        print("running canary eval")
        canary_loss, fitting_loss = get_fit_canary_loss(model, fitting_canaries_ids, canary_ids)  
        # print(canary_loss, fitting_loss)
        exposure = get_exposure(fitting_loss, canary_loss)
        print('Exposure :', exposure)
        
    if args.do_ref_model:
        model_ref.eval()
        losses_ref = []
        
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)

        nll, kl, outputs, outputs_noise = outputs[0], outputs[1], outputs[2], outputs[3]
        loss = nll + kl * args.beta
        # loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))

        if args.do_ref_model:
            with torch.no_grad():
                outputs_ref =model_ref(**batch)
            loss_ref = outputs_ref.loss
            losses_ref.append(accelerator.gather(loss_ref.repeat(args.per_device_eval_batch_size)))
   
    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]

    if args.do_ref_model:
        losses_ref = torch.cat(losses_ref)
        losses_ref = losses_ref[: len(eval_dataset)]
        sorted_ratio = sorted([l-l_ref for l,l_ref in zip (losses, losses_ref)])

    sorted_loss = sorted(losses)
    
    if args.do_ref_model:
        threshold_ref = sorted_ratio[int(0.1*len(sorted_ratio))]
        threshold = sorted_loss[int(0.1*len(losses))]
        if accelerator.is_local_main_process:
            print("threshold_ref is: " , threshold_ref.detach().item())
            print("threshold is: " , threshold.detach().item())
    else:
        threshold = sorted_loss[int(0.1*len(losses))]
        if accelerator.is_local_main_process:
            print("threshold is: " , threshold.detach().item())
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    ################################################    
    #run threshold on training samples
    losses = []
    if args.do_ref_model:
        model_ref.eval()
        losses_ref = []
        
    for step, batch in enumerate(tqdm(train_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        # loss = outputs.loss
        nll, kl, outputs, outputs_noise = outputs[0], outputs[1], outputs[2], outputs[3]
        loss = nll + kl * args.beta
        losses.append(accelerator.gather(loss.repeat(args.per_device_train_batch_size)))
        
        if args.do_ref_model:
            with torch.no_grad():
                outputs_ref = model_ref(**batch)
            loss_ref = outputs_ref.loss
            losses_ref.append(accelerator.gather(loss_ref.repeat(args.per_device_train_batch_size)))
          
    accelerator.wait_for_everyone()
    losses = torch.cat(losses)
    losses = losses[: len(train_dataset)]
    
    if args.do_ref_model:
        losses_ref = torch.cat(losses_ref)
        losses_ref = losses_ref[: len(train_dataset)]
        lr_rat = [l - l_r for l, l_r in zip(losses,losses_ref)]
        
    if args.do_ref_model:
        guess_cor = sum([1 for sample in losses if sample<threshold])
        guess_cor_ref =  sum([1 for sample in lr_rat if sample<threshold_ref])
    else:    
        guess_cor = sum([1 for sample in losses if sample<threshold])
    
    try:
        perplexity_train = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity_train = float("inf")
    #assert(len(losses)==len(lr_rat))
    if args.do_ref_model:
        print("correct cnt  ref is: " , guess_cor_ref, "all is: ", len(losses), "ratio is: ", guess_cor_ref/len(losses))
    print("correct cnt is: " , guess_cor, "all is: ", len(losses), "ratio is: ", guess_cor/len(losses))
    print(f"epoch {epoch}: perplexity: {perplexity} perplexity_train: {perplexity_train}")
    print("____")

    if args.do_ref_model:
        print(f"{guess_cor_ref/len(losses)}\n{guess_cor/len(losses)}\n{perplexity}\n{perplexity_train}")
        ratio = len(train_dataset)/len(eval_dataset)
        guess_cor_subsampled = sum([1 for sample in losses[::int(ratio)] if sample<threshold])
        guess_cor_ref_subsampled =  sum([1 for sample in lr_rat[::int(ratio)] if sample<threshold_ref])                
        print(f"{guess_cor_ref_subsampled/(len(lr_rat[::int(ratio)]))}\n{guess_cor_subsampled/len(losses[::int(ratio)])}\n{guess_cor_ref_subsampled/(guess_cor_ref_subsampled+int(0.1*len(eval_dataset)))}\n{guess_cor_subsampled/(guess_cor_subsampled+int(0.1*len(eval_dataset)))}")
    else:
        # print(f"{guess_cor/len(losses)}\n{perplexity}\n{perplexity_train}")
        print(f"Guess Correct :{guess_cor/len(losses)} perplexity : {perplexity} perplexity train : {perplexity_train}")
    
    if perplexity < best_val_perplexity and save_path is not None:
        best_val_perplexity = perplexity
        
        print(f"saved model! epoch {epoch}: perplexity: {best_val_perplexity}")
        torch.save(model.state_dict(), save_path)

    # print(f"perplexity : {perplexity} perplexity train : {perplexity_train}")

training epoch 1


100%|██████████| 566/566 [07:45<00:00,  1.22it/s]


NLL: 5.3600, KL: 929.5839


100%|██████████| 59/59 [00:20<00:00,  2.81it/s]


threshold is:  926.3203735351562


...

NLL: 3.9948, KL: 257.4106


100%|██████████| 59/59 [00:20<00:00,  2.82it/s]


threshold is:  262.3014831542969


100%|██████████| 566/566 [03:23<00:00,  2.78it/s]


correct cnt is:  20 all is:  2261 ratio is:  0.008845643520566122
epoch 2: perplexity: 1.795656892662619e+116 perplexity_train: 9.42584394424809e+115
____
Guess Correct :0.008845643520566122 perplexity : 1.795656892662619e+116 perplexity train : 9.42584394424809e+115
saved model! epoch 2: perplexity: 1.795656892662619e+116


In [30]:
# save_path = f'models/VariationalGPT2_wikitext.pt'
# torch.save(model.state_dict(), save_path)

In [31]:
import torch
# # Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_path = f'models/VariationalGPT2_wikitext.pt'
state_dict = torch.load(save_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [33]:
def generate(prompt, model, tokenizer, device, seed, max_length=100, temperature=0.7):
    input_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device)
    output = model.transformer.generate(input_ids, max_length=max_length, temperature=temperature, num_return_sequences=1, do_sample=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

prompt = 'the secret number is'
max_seq_len = 200
seed = 0
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, model, tokenizer, device, seed, max_seq_len, temperature)
    print(f'{str(temperature)}\n{generation}\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.5
the secret number is supposed to be the number of the number of the ship's number. The ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the ship's number in the following way : A ship's number is the number of the



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.7
the secret number is 00 00 02 – 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.75
the secret number is to be kept secret from the public, and the intelligence community has no way to verify that the number is genuine. " The military intelligence community and the intelligence community have neither the ability nor the will to verify the number. " If the number is not confirmed, an investigation is suspended, and the number of people targeted or killed is later included in the military intelligence collection. The government would not be able to provide a reliable account of the number of people killed, nor would it be able to estimate how many of those killed were civilians. The military intelligence community and the intelligence community have no way to verify the number of people killed. Intelligence officials at the Defense Department and the intelligence community are unable to confirm the number of people killed in the air strikes. The Air Force said there were no credible reports of civilian casualties of the strikes, which were conducted with the intent

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.8
the secret number is not the number of the game itself, but the computer, which has been " programmed in the early 80s ". The game is not the first game to use this system, but it is one of the few games still being played. The developers of the game have created another game using the secret numbers, such as Silent Hill's Silent Hill: The Secret Service. In June 2013, a sequel called Silent Hill: The Game Master Game that was originally developed for the PlayStation 4 ( PlayStation Portable ) was released. The sequel was named Silent Hill 4 : Game Master Game in Japan, and both games were released on the PlayStation Store and GameStop. The game was a remake of Silent Hill with new graphics, music, characters and the appearance of a new character, named " @-@ " (  The game, titled Silent Hill 4 : Game Master Game, released by Konami ) in Japan on June 1 2011. The game was released in Japan on August 18 2011,

1.0
the secret number is from Wikipedia " : The number can be found in En